# Convolutional Neural Network implementation
In CNN, the input image $X$ is of dimension $N\times C \times H \times W$, the filter $F$ is of dimension $F\times C \times HH \times WW$, the output $Y$ is of dimension $N\times F\times Hd \times Wd$. In this note, for simplicity reason, we look at one input example ($N=1$), assume one color layer of input ($C=1$), only one filter ($F=1$), thus all the input, filter, output are 2d matrices.


## Forward propogation
$X$ is of dimension $H \times W$, $F$ is of dimension $HH\times WW$, if the padding size is $P$, the stride is $S$, then the output $Y$ has dimension $Hd\times Wd$.
$$
Hd=1+(H+2P-HH)/S \\\
Wd=1+(W+2P-WW)/S
$$

$$
Y_{kl}=\sum_{i=0}^{HH-1} \sum_{j=0}^{WW-1} F_{ij} X_{i+kS, j+lS}
$$

Or we can write this equation in another form
$$
Y_{kl}=\sum_{m=kS}^{HH-1+kS} \sum_{n=lS}^{WW-1+lS} F_{m-kS,n-lS} X_{mn}
$$

If we make a very simple neural network, without hidden layer and activation function. The loss function is defined as
$$
L=\sum_k^{Hd-1} \sum_l^{Wd-1} f(Y_{kl})
$$
$f()$ is some defined function to calculate the total loss. 

Then the Gradient $L$ with respect to $X$ and $F$ are
$$
\begin{align}
\frac{\partial L}{\partial F_{ij}}&=\sum_k^{Hd-1} \sum_l^{Wd-1} f'(Y_{kl})\frac{\partial Y_{kl}}{\partial F_{ij}} \nonumber \\\
&=\sum_k^{Hd-1} \sum_l^{Wd-1} f'(Y_{kl})X_{i+kS,j+lS} \nonumber
\end{align}
$$

$$
\begin{align}
\frac{\partial L}{\partial X_{mn}}&=\sum_k^{Hd-1} \sum_l^{Wd-1} f'(Y_{kl})\frac{\partial Y_{kl}}{\partial X_{mn}} \nonumber \\\
&=\sum_k^{Hd-1} \sum_l^{Wd-1} f'(Y_{kl})F_{m-kS,n-lS} \nonumber
\end{align}
$$
in which, $f'(Y)$ is of dimension $Hd\times Wd$, same as $Y$.

## Where is convolution? Why $180^\circ$ rotation?

OK, for simplicity, let's assume there is no padding and stride is one ($P=0,S=1$).

Then we get $Hd=1+H-HH, Wd=1+W-WW$

$$
Y_{kl}=\sum_{i=0}^{HH-1} \sum_{j=0}^{WW-1} F_{ij} X_{i+k, j+l}
$$
This is actually the cross-correlation of $F$ and $X$, $F\star X$. Cross-correlation and convolution are quite similar to each other, in terms of calculation. We know that $\bar F \ast X = F \star X$, where $\bar F$ is the $180^\circ$ rotation of $F$. $180^\circ$ rotation is just another way to say 2D flip.

If we flip the Filter right-left and up-down, $\bar F_{HH-1-i,WW-1-j}=F_{ij}$, then after some rearrangement of the index, we get
$$
Y_{kl}=\sum_{i=0}^{HH-1} \sum_{j=0}^{WW-1} \bar F_{ij} X_{k-i+HH-1, l-j+WW-1}
$$
Which is exactly 2D convolution, $\bar F \ast X$.
And after careful check of the index of X, it never goes out of bound and exactly traverses from $0$ to $H-1$ for $k-i+HH-1$, $0$ to $W-1$ for $l-j+WW-1$, means that this convolution has mode of "Valid".

For gradient, we also have
$$
\frac{\partial L}{\partial F_{ij}}=\sum_k^{Hd-1} \sum_l^{Wd-1} f'(Y_{kl})X_{i+k,j+l} 
$$

$$
\frac{\partial L}{\partial X_{mn}}=\sum_k^{Hd-1} \sum_l^{Wd-1} f'(Y_{kl})F_{m-k,n-l}
$$

It is clear to see that $\frac{\partial L}{\partial F_{ij}} = \bar f'(Y) \ast X $ in "Valid" mode and $\frac{\partial L}{\partial X_{mn}} = f'(Y) \ast F $ in "Full" mode. $\bar f'(Y)$ is the $180^\circ$ rotation of $f(Y)$.

>Note that when $S>1$, it is no longer standard cross-correlation of convolution. But I guess it might be a convention to call this method Convolutional Neural Network. 

## A better way to calculate gradient
In calculating $\frac{\partial L}{\partial X_{mn}}$, altough the equation is clear, the index of $F$ will go out of bound. For the out of bound index of $F$, we need to set them to be zeros. This can be done by some ```if else``` check, or just pad lots of zeros to $F$ to form a new filter. And it will not be easy to figure out correctly with padding $P>0$ and strike $S>1$.

A trick to calculate $\frac{\partial L}{\partial X_{mn}}$ is that, instead of fixing the index of $X$, then figuring out the index of $f'(Y)$ , we fix the index of $f'(Y)$, then figure out the index of $X$. 

$$
\frac{\partial L}{\partial X_{i+kS,j+lS}}=\sum_k^{Hd-1} \sum_l^{Wd-1}f'(Y_{kl})F_{ij}
$$
The index $i,j$ will traverse the whole size of the filter $F$, and there is no need to concern about the index of $X$. This method is actually to "Think backward" of the forward propogation. This idea was inspired by discussion with Siyuan Wang. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import numpy as np
from scipy import signal

class MyException(Exception):
    pass

def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

def eval_numerical_gradient_array(f, x, df, h=1e-5):
  """
  Evaluate a numeric gradient for a function that accepts a numpy
  array and returns a numpy array.
  """
  grad = np.zeros_like(x)
  it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
  while not it.finished:
    ix = it.multi_index
    
    oldval = x[ix]
    x[ix] = oldval + h
    pos = f(x).copy()
    x[ix] = oldval - h
    neg = f(x).copy()
    x[ix] = oldval
    
    grad[ix] = np.sum((pos - neg) * df) / (2 * h)
    it.iternext()
  return grad

def conv_forward(x, w, b, conv_param,method='native'):
  """
  A naive implementation of the forward pass for a convolutional layer.

  The input consists of N data points, each with C channels, height H and width
  W. We convolve each input with F different filters, where each filter spans
  all C channels and has height HH and width HH.

  Input:
  - x: Input data of shape (N, C, H, W)
  - w: Filter weights of shape (F, C, HH, WW)
  - b: Biases, of shape (F,)
  - conv_param: A dictionary with the following keys:
    - 'stride': The number of pixels between adjacent receptive fields in the
      horizontal and vertical directions.
    - 'pad': The number of pixels that will be used to zero-pad the input.

  Returns a tuple of:
  - out: Output data, of shape (N, F, H', W') where H' and W' are given by
    H' = 1 + (H + 2 * pad - HH) / stride
    W' = 1 + (W + 2 * pad - WW) / stride
  - cache: (x, w, b, conv_param)
  """
  #############################################################################
  # TODO: Implement the convolutional forward pass.                           #
  # Hint: you can use the function np.pad for padding.                        #
  #############################################################################
  N,C,H,W=x.shape
  F,C,HH,WW=w.shape
  S=conv_param['stride']
  P=conv_param['pad']

  x_pad=np.pad(x,((0,0),(0,0),(P,P),(P,P)),'constant')

  Hout = 1 + int((H + 2 * P - HH) / S)
  Wout = 1 + int((W + 2 * P - WW) / S)

  out=np.empty((N,F,Hout,Wout))

  if method=='native':
    for i in range(N):
      for j in range(F):
        for k in range(Hout):
          for l in range(Wout):
            out[i,j,k,l]=np.sum(x_pad[i,:,k*S:HH+k*S,l*S:WW+l*S]*w[j,:,:,:])+b[j]

  elif method=='conv':
    if S!=1:
      raise MyException("In conv method, stride S has to be 1!")
  # Below is to demonstrate that CNN forward is indeed doing convolve2d (only if stride = 1)
    for i in range(N):
      for j in range(F):
        tmp=np.zeros((Hout,Wout))
        for k in range(C):
          tmp[:,:] +=signal.convolve2d(x_pad[i,k,:,:],w[j,k,::-1,::-1],'valid')
        out[i,j,:,:]=tmp+b[j]
  else:
    raise MyException("Only native and conv methods are allowed.")


  #############################################################################
  #                             END OF YOUR CODE                              #
  #############################################################################
  cache = (x, w, b, conv_param)
  return out, cache


def conv_backward(dout, cache,method='native'):
  """
  A naive implementation of the backward pass for a convolutional layer.

  Inputs:
  - dout: Upstream derivatives.
  - cache: A tuple of (x, w, b, conv_param) as in conv_forward_naive

  Returns a tuple of:
  - dx: Gradient with respect to x
  - dw: Gradient with respect to w
  - db: Gradient with respect to b
  """
  #############################################################################
  # TODO: Implement the convolutional backward pass.                          #
  #############################################################################
  x, w, b, conv_param = cache

  N,C,H,W=x.shape
  F,C,HH,WW=w.shape
  S=conv_param['stride']
  P=conv_param['pad']

  x_pad=np.pad(x,((0,0),(0,0),(P,P),(P,P)),'constant')

  _,_,Hout,Wout=dout.shape

  # out=np.empty((N,F,Hout,Wout))
  dx_pad=np.zeros(x_pad.shape)
  dw=np.empty(w.shape)
  db=np.empty(b.shape)

  if method=='native':
    for i in range(F):
      for j in range(C):
        for k in range(HH):
          for l in range(WW):
            dw[i,j,k,l]=np.sum(x_pad[:,j,k:k+S*Hout:S,l:l+S*Wout:S]*dout[:,i,:,:])
      db[i]=np.sum(dout[:,i,:,:])

    for i in range(N):
      for j in range(F):
        for k in range(Hout):
          for l in range(Wout):
            ## NOTE! here is the better way to calculate dx_pad
            dx_pad[i,:,k*S:k*S+HH,l*S:l*S+WW] +=w[j,:,:,:]*dout[i,j,k,l]

  elif method=='conv':
    if S!=1:
      raise MyException("In conv method, stride S has to be 1!")
  # using convolve2d, (only if stride = 1)
    for j in range(F):
      for k in range(C):
        tmp=np.zeros((HH,WW))
        for i in range(N):
          tmp[:,:] +=signal.convolve2d(x_pad[i,k,:,:],dout[i,j,::-1,::-1],'valid')
        dw[j,k,:,:]=tmp
      db[j]=np.sum(dout[:,j,:,:])

    for i in range(N):
      for k in range(C):
        tmp=np.zeros((H+2,W+2))
        for j in range(F):
          tmp[:,:] +=signal.convolve2d(w[j,k,:,:],dout[i,j,:,:],'full')
        dx_pad[i,k,:,:]=tmp

  else:
    raise MyException("Only native and conv methods are allowed.")


  dx=dx_pad[:,:,P:H+P,P:W+P]

  #############################################################################
  #                             END OF YOUR CODE                              #
  #############################################################################
  return dx, dw, db

## Forward Propagation in CNN

In [3]:
x_shape = (2, 3, 4, 4)
w_shape = (3, 3, 4, 4)
x = np.linspace(-0.1, 0.5, num=np.prod(x_shape)).reshape(x_shape)
w = np.linspace(-0.2, 0.3, num=np.prod(w_shape)).reshape(w_shape)
b = np.linspace(-0.1, 0.2, num=3)

conv_param = {'stride': 2, 'pad': 1}
out_native, _ = conv_forward(x, w, b, conv_param,method='native')
correct_out = np.array([[[[[-0.08759809, -0.10987781],
                           [-0.18387192, -0.2109216 ]],
                          [[ 0.21027089,  0.21661097],
                           [ 0.22847626,  0.23004637]],
                          [[ 0.50813986,  0.54309974],
                           [ 0.64082444,  0.67101435]]],
                         [[[-0.98053589, -1.03143541],
                           [-1.19128892, -1.24695841]],
                          [[ 0.69108355,  0.66880383],
                           [ 0.59480972,  0.56776003]],
                          [[ 2.36270298,  2.36904306],
                           [ 2.38090835,  2.38247847]]]]])

# Compare your output to ours; difference should be around 1e-8
print('Testing conv_forward')
print('difference: ', rel_error(out_native, correct_out))

Testing conv_forward
difference:  2.21214764175e-08


In [4]:
x_shape = (2, 3, 4, 4)
w_shape = (3, 3, 4, 4)
x = np.linspace(-0.1, 0.5, num=np.prod(x_shape)).reshape(x_shape)
w = np.linspace(-0.2, 0.3, num=np.prod(w_shape)).reshape(w_shape)
b = np.linspace(-0.1, 0.2, num=3)

conv_param = {'stride': 1, 'pad': 1}
out_native, _ = conv_forward(x, w, b, conv_param,method='native')
out_conv, _ = conv_forward(x, w, b, conv_param,method='conv')

# The difference between results from conv and native method should be around float64 machine epsilon, 1e-16'
print('Testing conv_forward native VS convolve2d')
print('difference: ', rel_error(out_native, out_conv))

Testing conv_forward native VS convolve2d
difference:  1.50175938895e-16


## Backpropagation in CNN

In [5]:
x = np.random.randn(4, 3, 7, 7)
w = np.random.randn(2, 3, 3, 3)
b = np.random.randn(2,)
dout = np.random.randn(4, 2, 4,4)
conv_param = {'stride': 2, 'pad': 1}

dx_num = eval_numerical_gradient_array(lambda x: conv_forward(x, w, b, conv_param,method='native')[0], x, dout)
dw_num = eval_numerical_gradient_array(lambda w: conv_forward(x, w, b, conv_param,method='native')[0], w, dout)
db_num = eval_numerical_gradient_array(lambda b: conv_forward(x, w, b, conv_param,method='native')[0], b, dout)

out, cache = conv_forward(x, w, b, conv_param,method='native')
dx, dw, db = conv_backward(dout, cache,method='native')

# Your errors should be around 1e-9'
print('Testing conv_backward function')
print('dx error: ', rel_error(dx, dx_num))
print('dw error: ', rel_error(dw, dw_num))
print('db error: ', rel_error(db, db_num))

Testing conv_backward function
dx error:  4.60052935193e-09
dw error:  1.7534740975e-09
db error:  1.77412698523e-11


In [6]:
x = np.random.randn(4, 3, 7, 7)
w = np.random.randn(2, 3, 3, 3)
b = np.random.randn(2,)
dout = np.random.randn(4, 2, 7,7)
conv_param = {'stride': 1, 'pad': 1}

out, cache = conv_forward(x, w, b, conv_param,method='native')
dx_conv, dw_conv, db_conv = conv_backward(dout, cache,method='conv')
dx_native, dw_native, db_native = conv_backward(dout, cache,method='native')

# The difference between results from conv and native method should be around float64 machine epsilon, 1e-16'
print('Testing conv_backward function native VS convolve2d')
print('dx error: ', rel_error(dx_conv, dx_native))
print('dw error: ', rel_error(dw_conv, dw_native))
print('db error: ', rel_error(db_conv, db_native))

Testing conv_backward function native VS convolve2d
dx error:  6.7191408824e-15
dw error:  1.21654565198e-15
db error:  0.0
